<a href="https://colab.research.google.com/github/Elmira-Rakhmetova/ML-Audit-project/blob/main/MLMNC_Group_Project_Team_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLMNC Group Project - Team 6

# 0 - Import dependencies and Audit Dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
audit_risk = pd.read_csv('https://raw.githubusercontent.com/Elmira-Rakhmetova/ML-Audit-project/main/audit_risk.csv')
trial = pd.read_csv('https://raw.githubusercontent.com/Elmira-Rakhmetova/ML-Audit-project/main/trial.csv')

# 1 - Data Exploration

In [ ]:
# how does the data look like?
trial.head()

,Sector_score,LOCATION_ID,PARA_A,SCORE_A,PARA_B,SCORE_B,TOTAL,numbers,Marks,Money_Value,MONEY_Marks,District,Loss,LOSS_SCORE,History,History_score,Score,Risk
0,3.89,23,4.18,6,2.50,2,6.68,5.0,2,3.38,2,2,0,2,0,2,2.4,1
1,3.89,6,0.00,2,4.83,2,4.83,5.0,2,0.94,2,2,0,2,0,2,2.0,0
2,3.89,6,0.51,2,0.23,2,0.74,5.0,2,0.00,2,2,0,2,0,2,2.0,0
3,3.89,6,0.00,2,10.80,6,10.80,6.0,6,11.75,6,2,0,2,0,2,4.4,1
4,3.89,6,0.00,2,0.08,2,0.08,5.0,2,0.00,2,2,0,2,0,2,2.0,0


In [ ]:
# brief data description 
trial.describe()

,Sector_score,PARA_A,SCORE_A,PARA_B,SCORE_B,TOTAL,numbers,Marks,Money_Value,MONEY_Marks,District,Loss,LOSS_SCORE,History,History_score,Score,Risk
count,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,775.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000
mean,20.184536,2.450194,3.512887,10.799988,3.131443,13.218481,5.067655,2.237113,14.137631,2.909794,2.505155,0.029639,2.061856,0.104381,2.167526,2.702577,0.626289
std,24.319017,5.678870,1.740549,50.083624,1.698042,51.312829,0.264449,0.803517,66.606519,1.597452,1.228678,0.184280,0.375080,0.531031,0.679869,0.858923,0.484100
min,1.850000,0.000000,2.000000,0.000000,2.000000,0.000000,5.000000,2.000000,0.000000,2.000000,2.000000,0.000000,2.000000,0.000000,2.000000,2.000000,0.000000
25%,2.370000,0.210000,2.000000,0.000000,2.000000,0.537500,5.000000,2.000000,0.000000,2.000000,2.000000,0.000000,2.000000,0.000000,2.000000,2.000000,0.000000
50%,3.890000,0.875000,2.000000,0.405000,2.000000,1.370000,5.000000,2.000000,0.090000,2.000000,2.000000,0.000000,2.000000,0.000000,2.000000,2.400000,1.000000
75%,55.570000,2.480000,6.000000,4.160000,4.000000,7.707500,5.000000,2.000000,5.595000,4.000000,2.000000,0.000000,2.000000,0.000000,2.000000,3.250000,1.000000
max,59.850000,85.000000,6.000000,1264.630000,6.000000,1268.910000,9.000000,6.000000,935.030000,6.000000,6.000000,2.000000,6.000000,9.000000,6.000000,5.200000,1.000000


## Correlations to identify dependent variables
What are the correlations between the variables? 

We use standard correlation coefficient and Spearman rank correlation

In [ ]:
trial.corr(method='pearson').style.format('{:.2f}').background_gradient()

,Sector_score,PARA_A,SCORE_A,PARA_B,SCORE_B,TOTAL,numbers,Marks,Money_Value,MONEY_Marks,District,Loss,LOSS_SCORE,History,History_score,Score,Risk
Sector_score,1.00,-0.22,-0.43,-0.13,-0.22,-0.15,-0.15,-0.17,-0.12,-0.32,-0.11,-0.08,-0.09,-0.11,-0.14,-0.34,-0.37
PARA_A,-0.22,1.00,0.50,0.16,0.36,0.27,0.13,0.14,0.45,0.29,0.13,0.05,0.04,0.12,0.17,0.43,0.29
SCORE_A,-0.43,0.50,1.00,0.25,0.57,0.30,0.24,0.27,0.21,0.48,0.09,0.09,0.09,0.18,0.27,0.72,0.67
PARA_B,-0.13,0.16,0.25,1.00,0.35,0.99,0.21,0.23,0.13,0.31,0.08,0.04,0.04,0.20,0.32,0.40,0.16
SCORE_B,-0.22,0.36,0.57,0.35,1.00,0.38,0.28,0.31,0.21,0.57,-0.00,0.10,0.09,0.20,0.31,0.90,0.52
TOTAL,-0.15,0.27,0.30,0.99,0.38,1.00,0.22,0.24,0.17,0.34,0.09,0.05,0.05,0.21,0.33,0.43,0.19
numbers,-0.15,0.13,0.24,0.21,0.28,0.22,1.00,0.91,0.19,0.45,0.13,-0.00,0.04,0.20,0.21,0.50,0.20
Marks,-0.17,0.14,0.27,0.23,0.31,0.24,0.91,1.00,0.22,0.51,0.15,0.00,0.04,0.23,0.25,0.57,0.23
Money_Value,-0.12,0.45,0.21,0.13,0.21,0.17,0.19,0.22,1.00,0.39,0.03,0.02,0.03,0.08,0.11,0.29,0.16
MONEY_Marks,-0.32,0.29,0.48,0.31,0.57,0.34,0.45,0.51,0.39,1.00,0.08,0.12,0.13,0.25,0.33,0.76,0.44


In [ ]:
trial.corr(method='spearman').style.format('{:.2f}').background_gradient()

,Sector_score,PARA_A,SCORE_A,PARA_B,SCORE_B,TOTAL,numbers,Marks,Money_Value,MONEY_Marks,District,Loss,LOSS_SCORE,History,History_score,Score,Risk
Sector_score,1.00,-0.38,-0.38,-0.13,-0.10,-0.31,-0.07,-0.07,-0.17,-0.13,-0.15,-0.08,-0.09,-0.05,-0.05,-0.30,-0.37
PARA_A,-0.38,1.00,0.91,0.53,0.55,0.82,0.28,0.28,0.46,0.45,0.09,0.10,0.10,0.27,0.27,0.72,0.65
SCORE_A,-0.38,0.91,1.00,0.54,0.56,0.79,0.29,0.29,0.48,0.49,0.08,0.10,0.10,0.28,0.28,0.77,0.69
PARA_B,-0.13,0.53,0.54,1.00,0.83,0.85,0.32,0.32,0.58,0.55,-0.04,0.07,0.06,0.30,0.30,0.72,0.46
SCORE_B,-0.10,0.55,0.56,0.83,1.00,0.78,0.33,0.33,0.58,0.57,-0.00,0.10,0.09,0.32,0.32,0.82,0.53
TOTAL,-0.31,0.82,0.79,0.85,0.78,1.00,0.32,0.32,0.59,0.55,0.02,0.10,0.10,0.30,0.30,0.84,0.69
numbers,-0.07,0.28,0.29,0.32,0.33,0.32,1.00,1.00,0.42,0.51,0.14,0.03,0.05,0.23,0.23,0.46,0.24
Marks,-0.07,0.28,0.29,0.32,0.33,0.32,1.00,1.00,0.42,0.51,0.14,0.03,0.05,0.23,0.23,0.46,0.24
Money_Value,-0.17,0.46,0.48,0.58,0.58,0.59,0.42,0.42,1.00,0.80,0.06,0.12,0.13,0.31,0.31,0.67,0.46
MONEY_Marks,-0.13,0.45,0.49,0.55,0.57,0.55,0.51,0.51,0.80,1.00,0.08,0.10,0.12,0.36,0.36,0.70,0.45


## What we observe
1. the difference in the correlation coefficients computed with Pearson and Spearman methods indicates the non-linear relationships; 
1. The output variable (Risk) is positively correlated with all the input variables except for Sector_score;
1. Risk is highly correlated (Spearman coefficient > 0.40) with: PARA_A, SCORE_A, PARA_B, SCORE_B, TOTAL, Money_Value, MONEY_Marks and Score (highest correlation of 0.86);
1. As for the correlations between the inputs, we observe highest correlations (Spearman coefficient >= 0.80) between PARA_A and SCORE_A (0.91), PARA_B and SCORE_B (0.83), TOTAL and PARA_A (0.82), TOTAL and PARA_B (0.85), LOSS_SCORE and Loss (0.98), Marks and numbers (1.00), MONEY_Marks and Money_Value (0.80), History_score and History (1.00), Score and SCORE_B	(0.82), Score and TOTAL (0.84).

## What we conclude based on the correlation coefficients:
- some of the high correlation coefficients between the 2 inputs indicate that 1 attribute is derived from another one; 
- for this reason, we exlude from further analysis: SCORE_A (derived from PARA_A), SCORE_B (derived from PARA_B), 
- Marks (derived from numbers), MONEY_Marks (derived from Money_Value), LOSS_SCORE (derived from Loss), History_score (derived from History)

## List of values considered for further analysis: 
- **LOCATION_ID**: Unique ID of the city/province
- **Sector_score**: Historical risk score value of the sector; 
- **PARA_A**: Discrepancy found in the plannedexpenditure of inspection and summary report A in Rs (in crore)
- **PARA_B**: Discrepancy found in the unplannedexpenditure of inspection and summary report B in Rs (in crore)
- **TOTAL**: Total amount of discrepancy found in other reports Rs (in crore)
- **numbers**: Historical discrepancy score
- **Money_Value**: Amount of money involved in misstatements in the past audits
- **District**: Historical risk score of a district in the last 10 years
- **Loss**: Amount of loss suffered by the firm last year
- **History**: Average historical loss suffered by firm in the last 10 years
- **Score**: Total risk score using analytical procedure
- **Risk** *(output variable)*: Risk Class assigned to an audit-case 

# 2 - Data preparation

## Cleaning the data



In [ ]:
trial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Sector_score   776 non-null    float64
 1   LOCATION_ID    776 non-null    object 
 2   PARA_A         776 non-null    float64
 3   SCORE_A        776 non-null    int64  
 4   PARA_B         776 non-null    float64
 5   SCORE_B        776 non-null    int64  
 6   TOTAL          776 non-null    float64
 7   numbers        776 non-null    float64
 8   Marks          776 non-null    int64  
 9   Money_Value    775 non-null    float64
 10  MONEY_Marks    776 non-null    int64  
 11  District       776 non-null    int64  
 12  Loss           776 non-null    int64  
 13  LOSS_SCORE     776 non-null    int64  
 14  History        776 non-null    int64  
 15  History_score  776 non-null    int64  
 16  Score          776 non-null    float64
 17  Risk           776 non-null    int64  
dtypes: float64

We observe `object` datatype in column "LOCATION ID"; as it is supposed to be a numerical variable, we have to exlude the rows with text:

In [ ]:
trial.LOCATION_ID.unique()

array(['23', '6', '7', '8', '13', '37', '24', '3', '4', '14', '5', '20',
       '19', '21', '22', '9', '11', '12', '29', '30', '38', '31', '2',
       '32', '16', '33', '15', '36', '34', '18', '25', '39', '27', '35',
       '40', '41', '42', '1', '28', 'LOHARU', 'NUH', 'SAFIDON', '43',
       '44', '17'], dtype=object)

In [ ]:
trial = trial[(trial.LOCATION_ID != 'LOHARU')]
trial = trial[(trial.LOCATION_ID != 'NUH')]
trial = trial[(trial.LOCATION_ID != 'SAFIDON')]
trial = trial.astype(float)

We need to make sure that there are no `nan` values within the dataset:

In [ ]:
# finding all nan values
print(np.isnan(trial).all())
print(np.argwhere(np.isnan(np.array(trial)))) # result shows that there is one single nan value in the dataset

# setting all nan values to 0
trial[np.isnan(trial)] = 0

Sector_score     False
LOCATION_ID      False
PARA_A           False
SCORE_A          False
PARA_B           False
SCORE_B          False
TOTAL            False
numbers          False
Marks            False
Money_Value      False
MONEY_Marks      False
District         False
Loss             False
LOSS_SCORE       False
History          False
History_score    False
Score            False
Risk             False
dtype: bool
[[639   9]]


## Selecting the data that we want to include
As discussed before, we want to include only a subset of the original dataset to get more accurate results.

As `LOCATION_ID` attribute does not hold any relevant information in terms of identifying whether a firm is potentially fraudulent or not, we will set it aside and do not incude in the classification model; however, we will use it in the end to cluster potentially fraudulent firms and see whether they tend to concentrate in some location(s)

In [ ]:
xnames = ['Sector_score', 'PARA_A','PARA_B', 'TOTAL', 'numbers', 'Money_Value', 'District', 'Loss', 'History', 'Score']
yname = 'Risk'

## Normalizing data

To get better results for our model, we need to normalize the input variables:

In [ ]:
# setting x and y
x = trial[xnames]
y = trial[yname].astype(int) # y values can only be 1 or 0, that is why they are changed into an integer

# converting values into a numpy array (not sure if this step is necessary)
x = np.array(x)
y = np.array(y)

In [ ]:
# this uses the keras normalizing method
x = tf.keras.utils.normalize(x)

## Splitting data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42) # train_test_split was imported in the beginning

## Final checkups
Here's a summary of what the data looks like now

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(541, 10)
(541,)
(232, 10)
(232,)


# 3 - Modelling and results



## 3.1 K_Neighbours

## 3.2 RandomForest (RF)

## 3.3 Neural Network (NN)

Sources:

https://www.tensorflow.org/tutorials/load_data/csv

In [ ]:

nn_model = tf.keras.models.Sequential([
tf.keras.layers.InputLayer(input_shape=(10)),
tf.keras.layers.Dense(2)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# making a few checks (not necessary anymore)
'''predictions = model(x_train[:1]).numpy()
print(f"(testing) Predictions are working: {predictions}")
print(f"(testing) Loss function is working: {loss_fn(y_train[:1], predictions).numpy()}")
print(f"The input shape of tf_x_train is: {x_train.shape}")
print(f"The input shape of tf_y_train is: {y_train.shape}\n")
print(f"Example entry of tf_x_train array: {x_train[0]}\n")
print(f"Example entry of tf_y_train array: {y_train[0]}\n"'''

nn_model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

result = nn_model.fit(x_train, y_train, epochs=1000)

Epoch 1/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.9077 - accuracy: 0.3216
Epoch 2/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.8928 - accuracy: 0.3309
Epoch 3/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.8790 - accuracy: 0.3383
Epoch 4/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.8652 - accuracy: 0.3549
Epoch 5/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.8519 - accuracy: 0.3734
Epoch 6/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.8391 - accuracy: 0.3974
Epoch 7/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.8266 - accuracy: 0.4067
Epoch 8/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.8143 - accuracy: 0.4233
Epoch 9/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.8023 - accuracy: 0.4381
Epoch 10/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.7909 - accuracy: 0.4492

In [ ]:
nn_model.weights

[<tf.Variable 'dense/kernel:0' shape=(10, 2) dtype=float32, numpy=
 array([[ 0.68425524, -0.5200589 ],
        [-7.9884806 ,  8.276195  ],
        [-0.9658108 ,  0.23555721],
        [-4.8452163 ,  4.5692935 ],
        [ 4.7841907 , -3.9040546 ],
        [-1.9996339 ,  0.91725874],
        [-4.4533067 ,  4.9167485 ],
        [-5.6294565 ,  5.4677114 ],
        [-4.17013   ,  5.0393515 ],
        [ 0.5858121 ,  0.2954777 ]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(2,) dtype=float32, numpy=array([-0.2868709 ,  0.28687114], dtype=float32)>]

In [ ]:
nn_model.evaluate(x_test,  y_test, verbose=2)

8/8 - 0s - loss: 0.2418 - accuracy: 0.9009


[0.24178344011306763, 0.9008620977401733]

## 3.4 Deep Neural Network (DNN)

In [ ]:
dnn_model = tf.keras.models.Sequential([
tf.keras.layers.InputLayer(input_shape=(10)),
tf.keras.layers.Dense(50, activation='relu'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(20, activation='relu'),
tf.keras.layers.Dense(2)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

dnn_model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

result = dnn_model.fit(x_train, y_train, epochs=1000)

Epoch 1/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.6374 - accuracy: 0.6525
Epoch 2/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.5767 - accuracy: 0.6932
Epoch 3/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.5244 - accuracy: 0.7375
Epoch 4/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.4765 - accuracy: 0.7523
Epoch 5/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.4323 - accuracy: 0.8115
Epoch 6/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.3946 - accuracy: 0.8244
Epoch 7/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.3876 - accuracy: 0.8133
Epoch 8/1000
17/17 [==============================] - 0s 3ms/step - loss: 0.3605 - accuracy: 0.8281
Epoch 9/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.3455 - accuracy: 0.8558
Epoch 10/1000
17/17 [==============================] - 0s 2ms/step - loss: 0.3265 - accuracy: 0.8669

In [ ]:
dnn_model.evaluate(x_test,  y_test, verbose=2)

8/8 - 0s - loss: 0.0044 - accuracy: 1.0000


[0.004391608759760857, 1.0]

## 3.5 Bayesian Net

## 3.6 J48

# 4 - Results

# 5 - Final remarks

# 6 - Appendix (and code that is not needed anymore but that might be useful to the team)